In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/sujin/PycharmProjects/verl")

import yaml

from verl.utils.dataset import SFTDataset

/root/miniconda3/envs/verl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-29 09:26:10,249	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
config_path = "/sujin/PycharmProjects/verl/verl/trainer/config/sft_trainer.yaml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)["data"]

parquet_files = "/sujin/PycharmProjects/verl/data/gsm8k/train.parquet"
tokenizer = "/sujin/Models/Qwen/Qwen2.5-0.5B-Instruct"

dataset = SFTDataset(parquet_files, tokenizer, config)

In [7]:
# data = dataset.__getitem__(2)
# for k, v in data.items():
#     # print(f"{k}: {v.shape}")
#     pass
print(dataset.tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

# Training effect evaluation

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/sujin/Models/Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [28]:
prompt = "John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home. He tries to get home in 4 hours but spends the first 2 hours in standstill traffic. He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph. How far is he from home at the end of those 4 hours?"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

To determine how far John is from his home at the end of the 4 hours, we need to calculate his total distance traveled and subtract this from his starting point.

1. **Calculate the distance driven on the way to home:**
   - Distance = Speed × Time
   - Distance = \( 60 \text{ mph} \times 3 \text{ hours} = 180 \text{ miles} \)

2. **Calculate the distance driven during the second part of the journey:**
   - Time spent in standstill traffic = 2 hours
   - Distance = Speed × Time
   - Distance = \( 30 \text{ mph} \times 2 \text{ hours} = 60 \text{ miles} \)
   - Total distance so far (including standstill) = \( 180 \text{ miles} + 60 \text{ miles} = 240 \text{ miles} \)

3. **Calculate the distance driven while traveling back home:**
   - Time taken to travel back home = 4 hours
   - Distance = Speed × Time
   - Distance = \( 80 \text{ mph} \times 4 \text{ hours} = 320 \text{ miles} \)
   - Total distance traveled so far = \( 240 \text{ miles} + 320 \text{ miles} = 560 \text{ miles} \)



In [29]:
model_name = "/tmp/sft_model/global_step_116"

tuned_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [30]:
prompt = "John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home. He tries to get home in 4 hours but spends the first 2 hours in standstill traffic. He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph. How far is he from home at the end of those 4 hours?"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = tuned_model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

He drove for 3+2=<<3+2=5>>5 hours.
His total distance was 60*5=<<60*5=300>>300 miles.
The first 2 hours were spent driving without any stops, so he had driven 300-100=<<300-100=200>>200 miles.
The next hour was spent driving at 30mph, so he traveled an additional 30*1/2=<<30*1/2=15>>15 miles.
Then he drove for another 2 hours at 80mph, which means he traveled an additional 80*2=<<80*2=160>>160 miles.
So his final distance from home was 300-200+15+160=<<300-200+15+160=245>>245 miles.
#### 245
